In [ ]:
# -*- coding: utf-8 -*-

"""
Created on Fri Jul 10 23:46:26 2020

@author: Elise Nguyen
"""
# Set path if need be
import os
path = "C:/Users/Elise Nguyen/Pension Board/Volatility-Study/doc"
os.chdir(path)

# Import required packages
import numpy as np
import pandas as pd
import pandas_datareader.data as web
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
import lxml.html as lh
from datetime import date
import re

#Pull data from URLs below using requests and lxml
today = date.today()

url='http://www.cboe.com/trading-tools/strategy-planning-tools/term-structure-data#3'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')
[len(T) for T in tr_elements[:12]]
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    #print('%d:"%s"'%(i,name))
    col.append((name,[]))
    
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 4, the //tr data is not from our table 
    if len(T)!=4:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
        
# Process data and Store dataframe in CBOE VIX file
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

regex = re.compile(r'[\n\r\t]')
df['Expiration Date'] = [regex.sub("", x) for x in df['Expiration Date']]
df['VIX'] = [regex.sub("", x) for x in df['VIX']]
df.to_csv('../output/CBOE VIX.csv', index = False, mode='a', header=False)

#Pull historical yahoo vix data

#yahoo_vix = web.DataReader(name = "^VIX", data_source = 'yahoo', start = '1990-01-01', end = '2020-07-08')
#yahoo_vix.to_csv(r'Yahoo VIX.csv', index = True)

#Update new yahoo vix data
yahoo_vix_update = web.DataReader(name = "^VIX", data_source = 'yahoo', start = '2020-07-09', end = today)
yahoo_vix_update.to_csv(r'Yahoo VIX.csv', index = True, mode='a',header = False )

#Pull data from cboe vol index vix futures
url='http://www.cboe.com/products/futures/vx-cboe-volatility-index-vix-futures'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')
[len(T) for T in tr_elements[:12]]
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    #print('%d:"%s"'%(i,name))
    col.append((name,[]))
    
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 8, the //tr data is not from our table 
    if len(T)!=8:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

# Store data frame in CBOE VIX futures file
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

df.to_csv('../output/CBOE VIX futures.csv', index = False, mode='a', header=False)